In [69]:
import subprocess
import numpy as np
import cv2
import os
from pathlib import Path
from PIL import Image

# === CONFIG ===
input_dir = Path(r"D:\notebook\car\captures\s_09_front_left-open__rear_right-open")
output_dir = Path(r"D:\notebook\car\captures_cropped\s_09_front_left-open__rear_right-open")
ffmpeg_path = r"C:\ffmpeg\bin\ffmpeg.exe"
resize_to = (448, 448)
trim_start = 1.0
threshold = 3
show_preview = True

output_dir.mkdir(parents=True, exist_ok=True)
video_files = list(input_dir.rglob("*.mp4"))

if not video_files:
    raise RuntimeError(f"❌ No videos found in {input_dir}")

print(f"🎥 Found {len(video_files)} video(s) in {input_dir}")

def extract_frames(video_path, temp_dir, fps=1):
    """Extract sample frames using ffmpeg (1 frame per second)."""
    temp_dir.mkdir(parents=True, exist_ok=True)
    frame_pattern = str(temp_dir / "frame_%03d.png")
    cmd = [
        ffmpeg_path, "-y",
        "-i", str(video_path),
        "-vf", f"fps={fps}",
        frame_pattern
    ]
    subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    return list(temp_dir.glob("frame_*.png"))

def detect_crop_from_frames(frames):
    all_coords = []
    for f in frames:
        img = cv2.imread(str(f))
        if img is None:
            continue
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray, (5, 5), 0)
        edges = cv2.Canny(blur, 5, 30)
        mask = (gray > threshold) | (edges > 0)
        coords = np.argwhere(mask)
        if coords.size > 0:
            all_coords.append(coords)
    if not all_coords:
        raise RuntimeError("❌ No valid non-black area detected.")
    coords = np.vstack(all_coords)
    y_min, x_min = coords.min(axis=0)
    y_max, x_max = coords.max(axis=0)
    w, h = x_max - x_min, y_max - y_min
    crop_filter = f"crop={w}:{h}:{x_min}:{y_min}"
    return (x_min, y_min, x_max, y_max, crop_filter)

# === Process all videos ===
for idx, video_path in enumerate(video_files, start=1):
    rel_path = video_path.relative_to(input_dir)
    output_path = output_dir / rel_path
    output_path.parent.mkdir(parents=True, exist_ok=True)

    print(f"\n[{idx}/{len(video_files)}] 🎬 Processing: {rel_path}")
    temp_dir = Path("temp_frames")
    for f in temp_dir.glob("*.png"):
        f.unlink(missing_ok=True)

    try:
        frames = extract_frames(video_path, temp_dir)
        if not frames:
            raise RuntimeError("⚠️ No frames extracted — video unreadable?")

        x_min, y_min, x_max, y_max, crop_str = detect_crop_from_frames(frames)
        print(f"🎯 Crop detected: {crop_str}")

        # Apply crop + resize + trim with ffmpeg
        cmd = [
            ffmpeg_path,
            "-y",
            "-ss", str(trim_start),
            "-i", str(video_path),
            "-vf", f"{crop_str},scale={resize_to[0]}:{resize_to[1]}",
            "-an",
            str(output_path)
        ]
        result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        if result.returncode == 0:
            print(f"✅ Saved cropped video: {output_path}")
        else:
            print(f"⚠️ FFmpeg failed:\n{result.stderr[:200]}")

        if idx == 1 and show_preview:
            img = cv2.imread(str(frames[len(frames)//2]))
            cv2.rectangle(img, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
            Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)).show()

    except Exception as e:
        print(f"❌ Failed to process {video_path}: {e}")

print("\n🎉 All videos processed successfully!")

🎥 Found 6 video(s) in D:\notebook\car\captures\s_09_front_left-open__rear_right-open

[1/6] 🎬 Processing: eagle_s_09_front_left-open__rear_right-open_tilt00.mp4
🎯 Crop detected: crop=684:287:137:18
✅ Saved cropped video: D:\notebook\car\captures_cropped\s_09_front_left-open__rear_right-open\eagle_s_09_front_left-open__rear_right-open_tilt00.mp4

[2/6] 🎬 Processing: eagle_s_09_front_left-open__rear_right-open_tilt01.mp4
🎯 Crop detected: crop=690:312:134:33
✅ Saved cropped video: D:\notebook\car\captures_cropped\s_09_front_left-open__rear_right-open\eagle_s_09_front_left-open__rear_right-open_tilt01.mp4

[3/6] 🎬 Processing: eagle_s_09_front_left-open__rear_right-open_tilt02.mp4
🎯 Crop detected: crop=694:378:132:9
✅ Saved cropped video: D:\notebook\car\captures_cropped\s_09_front_left-open__rear_right-open\eagle_s_09_front_left-open__rear_right-open_tilt02.mp4

[4/6] 🎬 Processing: eagle_s_09_front_left-open__rear_right-open_tilt03.mp4
🎯 Crop detected: crop=697:443:131:0
✅ Saved cropped vi

In [7]:
import subprocess
import numpy as np
import cv2
import os
from pathlib import Path
from PIL import Image

# === CONFIG ===
source_root = Path(r"D:\notebook\car\captures")
target_root = Path(r"D:\notebook\car\captures_cropped")
ffmpeg_path = r"C:\ffmpeg\bin\ffmpeg.exe"

resize_to = (448, 448)
trim_start = 1.0
threshold = 3
show_preview = True

# ensure target root exists
target_root.mkdir(parents=True, exist_ok=True)

def extract_frames(video_path, temp_dir, fps=1):
    """Extract sample frames using ffmpeg (1 frame per second)."""
    temp_dir.mkdir(parents=True, exist_ok=True)
    frame_pattern = str(temp_dir / "frame_%03d.png")
    cmd = [
        ffmpeg_path, "-y",
        "-i", str(video_path),
        "-vf", f"fps={fps}",
        frame_pattern
    ]
    subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    return list(temp_dir.glob("frame_*.png"))

def detect_crop_from_frames(frames):
    all_coords = []
    for f in frames:
        img = cv2.imread(str(f))
        if img is None:
            continue
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray, (5, 5), 0)
        edges = cv2.Canny(blur, 5, 30)
        mask = (gray > threshold) | (edges > 0)
        coords = np.argwhere(mask)
        if coords.size > 0:
            all_coords.append(coords)
    if not all_coords:
        raise RuntimeError("❌ No valid non-black area detected.")
    coords = np.vstack(all_coords)
    y_min, x_min = coords.min(axis=0)
    y_max, x_max = coords.max(axis=0)
    w, h = x_max - x_min, y_max - y_min
    crop_filter = f"crop={w}:{h}:{x_min}:{y_min}"
    return (x_min, y_min, x_max, y_max, crop_filter)

# === Loop through all folders in captures root ===
all_folders = [f for f in source_root.iterdir() if f.is_dir()]
print(f"📁 Found {len(all_folders)} folders in {source_root}")

for folder in all_folders:
    rel_folder = folder.name
    output_dir = target_root / rel_folder

    # Skip if already processed
    if output_dir.exists() and any(output_dir.rglob("*.mp4")):
        print(f"⏩ Skipping {rel_folder} — already cropped.")
        continue

    output_dir.mkdir(parents=True, exist_ok=True)

    # Collect all video files
    video_files = list(folder.rglob("*.mp4"))
    if not video_files:
        print(f"⚠️ No videos found in {rel_folder}")
        continue

    print(f"\n🎥 Processing folder: {rel_folder} ({len(video_files)} videos)")

    for idx, video_path in enumerate(video_files, start=1):
        rel_path = video_path.relative_to(folder)
        output_path = output_dir / rel_path
        output_path.parent.mkdir(parents=True, exist_ok=True)

        # Skip if output already exists
        if output_path.exists():
            print(f"⏭️ Skipping {rel_path} — already exists.")
            continue

        print(f"\n[{idx}/{len(video_files)}] 🎬 {rel_path}")
        temp_dir = Path("temp_frames")
        for f in temp_dir.glob("*.png"):
            f.unlink(missing_ok=True)

        try:
            frames = extract_frames(video_path, temp_dir)
            if not frames:
                raise RuntimeError("⚠️ No frames extracted — video unreadable?")

            x_min, y_min, x_max, y_max, crop_str = detect_crop_from_frames(frames)
            print(f"🎯 Crop detected: {crop_str}")

            # Apply crop + resize + trim with ffmpeg
            cmd = [
                ffmpeg_path,
                "-y",
                "-ss", str(trim_start),
                "-i", str(video_path),
                "-vf", f"{crop_str},scale={resize_to[0]}:{resize_to[1]}",
                "-an",
                str(output_path)
            ]
            result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            if result.returncode == 0:
                print(f"✅ Saved cropped video: {output_path}")
            else:
                print(f"⚠️ FFmpeg failed:\n{result.stderr[:200]}")

        

        except Exception as e:
            print(f"❌ Failed to process {video_path}: {e}")

print("\n🎉 All remaining folders processed successfully!")

📁 Found 32 folders in D:\notebook\car\captures
⏩ Skipping s_00_all_closed — already cropped.
⏩ Skipping s_01_front_left-open — already cropped.
⏩ Skipping s_02_front_right-open — already cropped.
⏩ Skipping s_03_front_left-open__front_right-open — already cropped.
⏩ Skipping s_04_rear_left-open — already cropped.
⏩ Skipping s_05_front_left-open__rear_left-open — already cropped.
⏩ Skipping s_06_front_right-open__rear_left-open — already cropped.
⏩ Skipping s_07_front_left-open__front_right-open__rear_left-open — already cropped.

🎥 Processing folder: s_08_rear_right-open (6 videos)

[1/6] 🎬 eagle_s_08_rear_right-open_tilt00.mp4
🎯 Crop detected: crop=684:287:137:18
✅ Saved cropped video: D:\notebook\car\captures_cropped\s_08_rear_right-open\eagle_s_08_rear_right-open_tilt00.mp4

[2/6] 🎬 eagle_s_08_rear_right-open_tilt01.mp4
🎯 Crop detected: crop=690:312:134:33
✅ Saved cropped video: D:\notebook\car\captures_cropped\s_08_rear_right-open\eagle_s_08_rear_right-open_tilt01.mp4

[3/6] 🎬 eagl